<a href="https://colab.research.google.com/github/shryu8902/KIRD_AUTOML/blob/main/KIRD_Day_2_3_%08AutoKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 오토케라스

autokeras는 keras 기반 오토ML 라이브러리로 딥러닝 모델 개발에 대한 오토ML 기능들을 제공합니다.

- Bayesian optimization 기반 NAS 수행
- 다양한 태스크에 대한 간단한 인터페이스 제공
- Image Classification/Regression
- Text Classification/Regression
- Structured Data Classification/Regression

In [1]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 66.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

import autokeras as ak

In [3]:
# 데이터 로드
data = load_diabetes()

# 훈련셋, 테스트셋 분리
x_train, x_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [5]:
# AutoKeras의 StructuredDataRegressor를 초기화합니다.
reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)

# 모델 훈련
reg.fit(x_train, y_train, epochs=5)

Trial 3 Complete [00h 00m 03s]
val_loss: 28125.994140625

Best val_loss So Far: 27199.484375
Total elapsed time: 00h 00m 10s
Epoch 1/5
12/12 [==============================] - 1s 5ms/step - loss: 29384.5352 - mean_squared_error: 29384.5352
Epoch 2/5
12/12 [==============================] - 0s 4ms/step - loss: 29140.5391 - mean_squared_error: 29140.5391
Epoch 3/5
12/12 [==============================] - 0s 4ms/step - loss: 28872.1836 - mean_squared_error: 28872.1836
Epoch 4/5
12/12 [==============================] - 0s 4ms/step - loss: 28551.2695 - mean_squared_error: 28551.2695
Epoch 5/5
12/12 [==============================] - 0s 4ms/step - loss: 28156.5859 - mean_squared_error: 28156.5859


In [6]:
# 모델 평가
print('Test Accuracy: ', reg.evaluate(x_test, y_test))


3/3 [==============================] - 0s 6ms/step - loss: 24871.9570 - mean_squared_error: 24871.9570
Test Accuracy:  [24871.95703125, 24871.95703125]


In [7]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 10)               0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 10)               21        
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                352       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

## 2. 블록연결을 통한 커스텀 서칭스페이스

In [19]:
# 입력 블록 생성
input_node = ak.StructuredDataInput()

# 블록 연결
processed_node = ak.StructuredDataBlock()(input_node)
latent_node = ak.DenseBlock(num_layers = 3,num_units=2)(processed_node)
output_node = ak.RegressionHead()(latent_node)

# AutoModel 초기화 및 훈련
auto_model = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=3)
auto_model.fit(x_train, y_train, epochs=3)

Trial 3 Complete [00h 00m 04s]
val_loss: 28192.810546875

Best val_loss So Far: 28192.810546875
Total elapsed time: 00h 00m 12s
Epoch 1/3
12/12 [==============================] - 2s 5ms/step - loss: 29696.3887 - mean_squared_error: 29696.3887
Epoch 2/3
12/12 [==============================] - 0s 4ms/step - loss: 29659.9062 - mean_squared_error: 29659.9062
Epoch 3/3
12/12 [==============================] - 0s 4ms/step - loss: 29623.7930 - mean_squared_error: 29623.7930


In [20]:
# 모델 평가
custom_model = auto_model.export_model()
custom_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 10)               0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 32)                352       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 re_lu_1 (ReLU)              (None, 32)                0         
                                                             

# Classification 모델
지난 모델 개발에 활용했던 타이타닉 데이터에 대해서 오토케라스로 분류모델을 만들어보겠습니다.

In [24]:
!wget https://raw.githubusercontent.com/shryu8902/KIRD_AUTOML/main/Titanic-Dataset.csv
titanic_data = pd.read_csv('./Titanic-Dataset.csv')

--2023-07-05 14:16:10--  https://raw.githubusercontent.com/shryu8902/KIRD_AUTOML/main/Titanic-Dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘Titanic-Dataset.csv’

Titanic-Dataset.csv 100%[===================>]  59.76K  --.-KB/s    in 0.008s  

2023-07-05 14:16:10 (7.53 MB/s) - ‘Titanic-Dataset.csv’ saved [61194/61194]



In [25]:
from sklearn.preprocessing import LabelEncoder

titanic_data.drop(['PassengerId','Cabin','Name','Ticket','Embarked'],axis = 1, inplace=True)
titanic_data.Age.fillna(-1, inplace = True)
titanic_data['Sex'] = LabelEncoder().fit_transform(titanic_data['Sex'])  # 'sex' feature를 numerical로 변환

In [26]:
X = titanic_data.drop('Survived', axis=1)
y = titanic_data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0) # Stratified split 적용


In [39]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare
502,3,0,-1.0,0,0,7.6292
464,3,1,-1.0,0,0,8.0500
198,3,0,-1.0,0,0,7.7500
765,1,0,51.0,1,0,77.9583
421,3,1,21.0,0,0,7.7333
...,...,...,...,...,...,...
131,3,1,20.0,0,0,7.0500
490,3,1,-1.0,1,0,19.9667
528,3,1,39.0,0,0,7.9250
48,3,1,-1.0,2,0,21.6792


In [46]:
# 정형데이터 모델에 대한 학습
# NAS 수행
reg = ak.StructuredDataClassifier(max_trials=10, tuner = 'bayesian',overwrite = True)

# 모델 훈련
reg.fit(X_train, y_train, epochs=5)

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.5288461446762085

Best val_accuracy So Far: 0.692307710647583
Total elapsed time: 00h 01m 06s
Epoch 1/5
23/23 [==============================] - 3s 8ms/step - loss: 0.7977 - accuracy: 0.6222
Epoch 2/5
23/23 [==============================] - 0s 7ms/step - loss: 0.6543 - accuracy: 0.6615
Epoch 3/5
23/23 [==============================] - 0s 7ms/step - loss: 0.5592 - accuracy: 0.7205
Epoch 4/5
23/23 [==============================] - 0s 7ms/step - loss: 0.5123 - accuracy: 0.7570
Epoch 5/5
23/23 [==============================] - 0s 6ms/step - loss: 0.4825 - accuracy: 0.7795


In [47]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 6)                0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 6)                13        
 n)                                                              
                                                                 
 dense (Dense)               (None, 1024)              7168      
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                             

In [48]:
# 최종 모델에대한 성능 평가
model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 4ms/step - loss: 0.6507 - accuracy: 0.6201


[0.6506813168525696, 0.6201117038726807]

In [49]:
# 결정된 인공신경망 구조를 바탕으로 추가학습 수행
model.fit(X_train, y_train, epochs = 20)

Epoch 1/20
23/23 [==============================] - 3s 7ms/step - loss: 0.4760 - accuracy: 0.7879
Epoch 2/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4734 - accuracy: 0.7907
Epoch 3/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4565 - accuracy: 0.7992
Epoch 4/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4261 - accuracy: 0.8160
Epoch 5/20
23/23 [==============================] - 0s 7ms/step - loss: 0.4484 - accuracy: 0.8006
Epoch 6/20
23/23 [==============================] - 0s 7ms/step - loss: 0.4361 - accuracy: 0.8090
Epoch 7/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4383 - accuracy: 0.8048
Epoch 8/20
23/23 [==============================] - 0s 10ms/step - loss: 0.4218 - accuracy: 0.8216
Epoch 9/20
23/23 [==============================] - 0s 10ms/step - loss: 0.4183 - accuracy: 0.8258
Epoch 10/20
23/23 [==============================] - 0s 9ms/step - loss: 0.4221 - accuracy: 0.8174
Epoch 11/20
23/23

In [50]:
# 추가학습 이후 성능 증가 확인가능
model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 4ms/step - loss: 0.4492 - accuracy: 0.8212


[0.4491870701313019, 0.8212290406227112]